# NCAA2024-中文糖尿病问题分类评测

## 赛题背景

赛题链接：https://tianchi.aliyun.com/competition/entrance/532176  
 中文糖尿病问题分类评测任务的目标是自动对患者提出的涉及糖尿病的问题进行分类。随着糖尿病成为全球重要的公共卫生挑战之一，互联网的迅速发展使得二型糖尿病患者和高危人群对专业信息的需求越来越迫切。因此，糖尿病自动问答服务在日常健康服务中的作用变得越来越重要。此任务的数据集包含了中文糖尿病问题分为6类。参与者需要对测试集中的糖尿病问题进行分类预测，并填补测试数据集中类别标签数据的空缺。评估过程将重点分析填充数据的误差，并给出预测性能得分。此外，本次任务也是NCAA 2024的评测任务之一，旨在提升搜索结果的准确性，推动糖尿病自动问答服务的发展。

## 赛题任务

NCAA 2024数据集基于NCAA 2023数据集进行补充增强。评测数据集包含的中文糖尿病问题一共分为6类，  
包括诊断、治疗、常识、健康生活方式、流行病学、其他。  
数据以 9：2：2 的比例划分为训练集、验证集和测试集。总计13000条数据。数据集都是以 .txt 格式存储。训练集、验证集和测试集包含question和label，分类数据集包含class和label。参赛者需要预测测试集中糖尿病问题对应的分类，预测完成后需将测试数据集空缺的类别标签数据进行填充。

## 评分标准

该任务使用准确率（Acc，Accuracy）作为整体排名标准，公式如下
$$
 \text{准确率（Accuracy）} = \frac{\text{预测正确样本数}}{\text{总样本数} } 
$$


# 导入相关包

In [154]:
import pandas as pd
import numpy as np
import jieba,re
from gensim import models, similarities
import gensim.corpora as corpora

import transformers
from transformers import AutoModel, AutoTokenizer,AutoConfig

from sklearn.model_selection import train_test_split
from gensim.models import word2vec
import matplotlib.pyplot as plt
import torch.nn.functional as F
import torch
from torch import nn
from transformers import BertTokenizerFast
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from tqdm.notebook import tqdm
import random
import os

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from torch.optim import AdamW,Adam
from transformers import  get_linear_schedule_with_warmup,get_cosine_schedule_with_warmup


# 读取源数据

In [155]:
# 读取训练集
train_df=pd.read_csv('train.txt', sep='\t', header=None)
valid_df=pd.read_csv('dev.txt', sep='\t', header=None)
test_df=pd.read_csv("test.txt", sep='\t', header=None)
train_df.columns = ['text', 'label'] 
valid_df.columns = ['text', 'label'] 
test_df.columns =['text'] 

In [156]:
len(test_df['text'])

2000

In [157]:
train_df.head()

,text,label
0,空腹血糖78，是否属于糖尿病范围?,0
1,减肥后是否能改善糖尿病状况?,3
2,患有糖尿病的母亲而父亲没有，是否会遗传给下一代?,2
3,糖尿病是否会引起眼睛水肿?,4
4,妊娠期糖尿病的注意事项是什么?,2


# CUDA

In [158]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

### 固定种子

In [159]:
SEED = 42
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
    
seed_everything(SEED)

# 数据观察

In [160]:
# 无空值
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    2000 non-null   object
dtypes: object(1)
memory usage: 15.8+ KB


查看各样本比例 

In [161]:
train_df.value_counts('label')

label
3    2723
1    2292
2    1707
4     897
0     747
5     634
Name: count, dtype: int64

In [162]:
test_df['text']

0                 空腹血糖5.8是否偏高?
1          2型糖尿病患者空腹血糖正常范围是多少?
2        1型糖尿病性周围神经病变会导致下肢疼痛吗?
3       42岁患有两年糖尿病史的男性可以食用海参吗?
4                 糖尿病患者可以饮用茶吗?
                 ...          
1995             经常吃糖是否会引起糖尿病?
1996             空腹血糖高是糖尿病病征吗?
1997           患了糖尿病很焦虑应该如何应对?
1998               糖尿病和智商有关系吗?
1999             糖尿病可以不注射胰岛素吗?
Name: text, Length: 2000, dtype: object

## 设置参数

## 选择预训练模型

In [163]:
tokenizer =AutoTokenizer.from_pretrained("chinese-roberta-wwm/")

## 定义MyDataset

In [164]:
class MyDataset(Dataset):  
    def __init__(self, data, tokenizer, mode='train', device='cpu'):  
        """  
        初始化数据集。  
        :param data: 包含评论和标签的DataFrame或其他类似数据结构。  
        :param tokenizer: 用于文本编码的分词器。  
        :param mode: 数据集模式，'train'、'test' 或其他。  
        :param device: 数据应被发送到的设备，通常是'cpu'或'cuda'（如果可用）。  
        """  
        self.data = data  
        self.tokenizer = tokenizer  
        self.mode = mode  
        self.device = device  
        # 检查CUDA是否可用，如果指定了'cuda'但不可用，则回退到'cpu'  
        if self.device == 'cuda' and not torch.cuda.is_available():  
            self.device = 'cpu'  
  
    def __len__(self):  
        """返回数据集的大小。"""  
        return len(self.data)  
      
    def __getitem__(self, index):  
        """  
        返回给定索引处的样本，并确保它在正确的设备上。  
        :param index: 样本索引。  
        """  
        # 获取并编码文本  
        text = self.data.loc[index, 'text']  
        encoded_text = self.tokenizer.encode_plus(  
            text,   
            truncation=True,  
            padding='max_length',  
            max_length=32,  
            return_tensors='pt'  
        )  
          
        input_ids = encoded_text['input_ids'].to(self.device).squeeze()  
        attention_mask = encoded_text['attention_mask'].to(self.device).squeeze()  
          
        # 根据模式返回样本  
        if self.mode == 'train':  
            label = self.data.loc[index, 'label']  
            label_tensor = torch.tensor(label, device=self.device)            
            return {  
                'input_ids': input_ids,  
                'attention_mask': attention_mask,  
                'label': label_tensor  
            }  
        else:   
            return {  
                'input_ids': input_ids,  
                'attention_mask': attention_mask  
            }  

## 划分数据集

In [165]:
train_data = train_df.reset_index(drop=True)
valid_data = valid_df.reset_index(drop=True)
test_data = test_df.copy()

print("len(train_data):",len(train_data))
print("len(valid_data):",len(valid_data))
print("len(test_data):",len(test_data))

len(train_data): 9000
len(valid_data): 2000
len(test_data): 2000


In [166]:
tokenizer

BertTokenizerFast(name_or_path='E:/NLP综合实践/阶段一/O2O商铺食品安全相关评论发现/chinese-roberta-wwm/', vocab_size=21128, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True)

In [167]:
# 调用 MyDataset
# 训练集
train_dataset = MyDataset(train_data, tokenizer=tokenizer,mode='train',device=device)
# 验证集
valid_dataset = MyDataset(valid_data, tokenizer=tokenizer,mode='train',device=device)
# 测试集
test_dataset = MyDataset(test_data, tokenizer=tokenizer,mode='test',device=device)

In [168]:
# 调用 dataloader
# 批处理大小
batch_size = 32
train_dataloader = DataLoader(train_dataset,batch_size = batch_size, shuffle=True)
valid_dataloader = DataLoader(valid_dataset,batch_size = batch_size, shuffle=False)
test_dataloader  = DataLoader(test_dataset,batch_size = batch_size, shuffle=False)

In [169]:
for val_batch in tqdm(valid_dataloader):
    print(val_batch)

  0%|          | 0/63 [00:00<?, ?it/s]

{'input_ids': tensor([[ 101, 5131, 2228,  ...,    0,    0,    0],
        [ 101, 5131, 2228,  ...,    0,    0,    0],
        [ 101,  123, 1798,  ...,    0,    0,    0],
        ...,
        [ 101,  123, 1798,  ...,    0,    0,    0],
        [ 101,  122, 1798,  ...,    0,    0,    0],
        [ 101, 2642, 3300,  ...,    0,    0,    0]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]], device='cuda:0'), 'label': tensor([4, 3, 3, 2, 3, 5, 3, 1, 3, 3, 1, 2, 0, 4, 2, 2, 2, 0, 1, 2, 4, 3, 3, 3,
        3, 2, 3, 1, 3, 4, 1, 3], device='cuda:0')}
{'input_ids': tensor([[ 101, 8203, 2259,  ...,    0,    0,    0],
        [ 101, 5131, 2228,  ...,    0,    0,    0],
        [ 101, 2642, 3300,  ...,    0,    0,    0],
        ...,
        [ 101, 6117, 5131,  ...,    0,    0,    0],
        [ 101

## 模型定义

## RoBerta

In [171]:
class MyBERT(nn.Module):
    def __init__(self, num_classes=6):
        super(MyBERT, self).__init__()
        # 加载 RoBERTa 预训练模型的配置
        config = AutoConfig.from_pretrained("chinese-roberta-wwm/")
        # 加载 RoBERTa 预训练模型
        self.bert = AutoModel.from_pretrained("chinese-roberta-wwm/", config=config)

        self.out = nn.Linear(config.hidden_size , num_classes)  

    def forward(self, input_ids, attention_mask):
        # 获取 RoBERTa 的输出
        bert_out = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask)
        # 获取 RoBERTa 的 Pooler 输出
        pooler = bert_out['pooler_output']
        # 通过分类层进行分类
        out = self.out(pooler)

        return out


In [172]:
# 实例化模型
model = MyBERT().to(device)
model_name='Roberta'

state_dict = model.state_dict()
for name, param in state_dict.items():
    if 'embedding' in name:
        print(name)

bert.embeddings.word_embeddings.weight
bert.embeddings.position_embeddings.weight
bert.embeddings.token_type_embeddings.weight
bert.embeddings.LayerNorm.weight
bert.embeddings.LayerNorm.bias


In [173]:
#print(model)

MyBERT(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tru

## 模型训练

In [176]:
#定义损失函数，优化器
num_epochs=20

#交叉熵损失函数
loss_fn=nn.CrossEntropyLoss()

#优化器
optimizer = Adam(model.parameters(), lr=5e-5)

total_steps=num_epochs * len(train_dataloader)

#学习率调度器
scheduler = get_cosine_schedule_with_warmup(optimizer, 
                                            num_training_steps=total_steps,
                                            num_warmup_steps=total_steps*0.1)

In [177]:
%%time

train_losses = []
valid_losses = []
Best_acc = 0
step = 0
loss_sum = 0

for epoch in tqdm(range(num_epochs)):
    model.train()  # 设置为训练模式
    for batch in tqdm(train_dataloader):
        step += 1

        # 正常训练
        out = model(batch['input_ids'], batch['attention_mask'])
        loss = loss_fn(out, batch['label'])
        loss_sum += loss.item()
        loss.backward()  # 正向传播

        optimizer.step()
        scheduler.step()

        optimizer.zero_grad()
        
        if step % 50 == 0:
            print(f"epoch：{epoch + 1}，平均训练损失：{loss_sum / 50}")
            loss_sum = 0
            
    # 验证集上进行评估 
    model.eval()  # 设置为评估模式
    valid_loss_sum = 0
    correct = 0
    total = 0
    preds = []
    labels = []

    with torch.no_grad():  # 禁用梯度计算
        for val_batch in tqdm(valid_dataloader):
            outputs = model(val_batch['input_ids'], val_batch['attention_mask'])
            predicted_labels = torch.argmax(outputs, 1)
            correct += (predicted_labels == val_batch['label']).sum()
            total += val_batch['label'].size(0)
            preds.extend(list(predicted_labels.cpu().numpy()))
            labels.extend(list(val_batch['label'].cpu().numpy()))
    
    accuracy = correct / total
    valid_losses.append(accuracy)
    # 检查是否是当前最佳准确率
    if Best_acc < accuracy:
        Best_acc = accuracy
        torch.save(model.state_dict(), f'{model_name}/model_{Best_acc}.bin')
    print(f"epoch：{epoch + 1}，验证集准确率：{accuracy}，最高准确率：{Best_acc}")
    

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/282 [00:00<?, ?it/s]

epoch：1，平均训练损失：1.349181468486786
epoch：1，平均训练损失：0.6432392776012421
epoch：1，平均训练损失：0.5381557792425156
epoch：1，平均训练损失：0.5547611939907074
epoch：1，平均训练损失：0.5264460629224778


  0%|          | 0/63 [00:00<?, ?it/s]

epoch：1，验证集准确率：0.7645000219345093，最高准确率：0.7645000219345093


...

epoch：4，验证集准确率：0.7690000534057617，最高准确率：0.7765000462532043


  0%|          | 0/282 [00:00<?, ?it/s]

epoch：5，平均训练损失：0.14081998787820338
epoch：5，平均训练损失：0.10697930347174406
epoch：5，平均训练损失：0.09748037463054061
epoch：5，平均训练损失：0.11912685234099626
epoch：5，平均训练损失：0.10019121013581753
epoch：5，平均训练损失：0.12270022213459014


  0%|          | 0/63 [00:00<?, ?it/s]

epoch：5，验证集准确率：0.7700000405311584，最高准确率：0.7765000462532043
CPU times: total: 3min 42s
Wall time: 5min 58s


In [178]:
loss.item()

0.010486189275979996

In [179]:
# 读取保存路径下的模型名称，并提取分数
scores = ['.'.join(model_name.split('_')[-1].split('.')[:-1]) for model_name in os.listdir(f"{model_name}/")]
# 获取最高分
Best_F1 = max(scores)
# 加载训练中保存的最高分模型
model = MyBERT().to(device)
model.load_state_dict(torch.load(f'{model_name}/model_{Best_F1}.bin', map_location='cpu'))

<All keys matched successfully>

In [185]:
# 开始模型推理
model.eval()
preds = [] 
with torch.no_grad():  # 禁用梯度计算
    for val_batch in tqdm(test_dataloader):
        outputs = model(val_batch['input_ids'], val_batch['attention_mask'])
        predicted_labels = torch.argmax(outputs, 1)
        preds.extend(list(predicted_labels.cpu().numpy()))

  0%|          | 0/63 [00:00<?, ?it/s]

In [186]:
# 创建结果 DataFrame 的副本，以避免 SettingWithCopyWarning
test_df.columns = ['text']
result = test_df.copy()
result['label'] = preds
result.to_csv('result.txt', sep='\t', index=False, header=False)

In [188]:
result

,text,label
0,空腹血糖5.8是否偏高?,0
1,2型糖尿病患者空腹血糖正常范围是多少?,0
2,1型糖尿病性周围神经病变会导致下肢疼痛吗?,4
3,42岁患有两年糖尿病史的男性可以食用海参吗?,3
4,糖尿病患者可以饮用茶吗?,3
...,...,...
1995,经常吃糖是否会引起糖尿病?,2
1996,空腹血糖高是糖尿病病征吗?,0
1997,患了糖尿病很焦虑应该如何应对?,1
1998,糖尿病和智商有关系吗?,2


In [189]:
torch.cuda.empty_cache()